In [1]:
locations={1000: 'Leeuwarden',
 1001: 'Drachten',
 1002: 'Sneek',
 1004: 'Harlingen',
 1005: 'Joure',
 1006: 'Wolvega',
 1007: 'Franeker',
 1008: 'Dokkum',
 1009: 'Lemmer',
 1010: 'Bolsward',
 1011: 'Burgum',
 1003: 'Heerenveen',
 160: None,
 199: "Afsluitdijk"}

In [2]:
for i in range(0,2**8):
    attempt="0"*(8-len(str(bin(i))[2:]))+str(bin(i))[2:]
    print(len(attempt),i)

8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
8 16
8 17
8 18
8 19
8 20
8 21
8 22
8 23
8 24
8 25
8 26
8 27
8 28
8 29
8 30
8 31
8 32
8 33
8 34
8 35
8 36
8 37
8 38
8 39
8 40
8 41
8 42
8 43
8 44
8 45
8 46
8 47
8 48
8 49
8 50
8 51
8 52
8 53
8 54
8 55
8 56
8 57
8 58
8 59
8 60
8 61
8 62
8 63
8 64
8 65
8 66
8 67
8 68
8 69
8 70
8 71
8 72
8 73
8 74
8 75
8 76
8 77
8 78
8 79
8 80
8 81
8 82
8 83
8 84
8 85
8 86
8 87
8 88
8 89
8 90
8 91
8 92
8 93
8 94
8 95
8 96
8 97
8 98
8 99
8 100
8 101
8 102
8 103
8 104
8 105
8 106
8 107
8 108
8 109
8 110
8 111
8 112
8 113
8 114
8 115
8 116
8 117
8 118
8 119
8 120
8 121
8 122
8 123
8 124
8 125
8 126
8 127
8 128
8 129
8 130
8 131
8 132
8 133
8 134
8 135
8 136
8 137
8 138
8 139
8 140
8 141
8 142
8 143
8 144
8 145
8 146
8 147
8 148
8 149
8 150
8 151
8 152
8 153
8 154
8 155
8 156
8 157
8 158
8 159
8 160
8 161
8 162
8 163
8 164
8 165
8 166
8 167
8 168
8 169
8 170
8 171
8 172
8 173
8 174
8 175
8 176
8 177
8 178
8 179
8 180
8 181
8 182
8 183
8 184


In [3]:
list_of_vals=[]
for i in range(0,2**7):
    basevals={160:20, 199:25, "Leeuwarden":66,"Drachten":36,"Sneek":27,"Harlingen":13.086859668020224,"Joure":10,"Wolvega":11,\
                    "Franeker":10,"Dokkum":8.980727539476826,"Lemmer":10,"Bolsward":9.477514803509319,"Burgum":10,"Heerenveen":24}
    attempt="0"*(7-len(str(bin(i))[2:]))+str(bin(i))[2:]
    if attempt[0]=="0":
        basevals["Leeuwarden"]=56
    if attempt[1]=="0":
        basevals["Lemmer"]=14
    if attempt[2]=="0":
        basevals["Dokkum"]=11
    if attempt[3]=="0":
        basevals[160]=12
    if attempt[4]=="0":
        basevals["Drachten"]=30
    if attempt[5]=="0":
        basevals["Sneek"]=35
    if attempt[6]=="0":
        for k in basevals.keys():
            if k==199: continue
            basevals[k]*=0.8
    list_of_vals.append((i,basevals))

In [4]:
vals=open("testvalues.txt","w")
for i in list_of_vals:
    vals.write(str(i)+"\n")
vals.close()

In [5]:
smoothing=1/10
runnr=89
# old_dist=34.63869707966813
stadnamen=['Leeuwarden', 'Drachten', 'Sneek', 'Heerenveen', 'Harlingen','Joure', 'Wolvega', 'Franeker', 'Dokkum', 'Lemmer', 'Bolsward','Burgum']
spawn_hoeveelheden={160:20, 199:25, "Leeuwarden":66,"Drachten":36,"Sneek":27,"Harlingen":13.086859668020224,"Joure":10,"Wolvega":11,\
                    "Franeker":10,"Dokkum":8.980727539476826,"Lemmer":10,"Bolsward":9.477514803509319,"Burgum":10,"Heerenveen":24}
import numpy as np
import networkx as nx
# import rustworkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from itertools import islice
# %matplotlib qt
import shapely
import matplotlib.patheffects as pe
import contextily as cx
def distribute(needs,space): #distributes cars, there might be a faster way, but i cannot think of one right now
    endresult=np.zeros(len(needs),dtype=int)
    # filter=needs
    while space>=np.sum((needs>0)):
        add=min(np.min(needs[needs>0]),space//np.sum((needs>0)))
        endresult[needs>0]+=add
        space-=add*np.sum((needs>0))
        needs[needs>0]-=add
    endresult[np.nonzero(needs>0)[0][:space]]+=1
    return endresult
class edge(object):
    def __init__(self,length,speed,lanes,laneboundaries=None,id=0):
        self.length = int(length)
        self.lanes = int(lanes)
        if laneboundaries==None: laneboundaries=(int(lanes),int(lanes))
        self.basespeed=int(speed)
        self.speed=int(speed)
        self.cars = np.zeros((int(lanes*length/(4.5+speed/1.8)),5)) #collums: occupied,age,remaining travel time, spawnpoint, destination
        self.C0=int(lanes*length/(4.5+speed/1.8))
        self.Cmax=int(laneboundaries[1]*length/(4.5+speed/1.8))
        self.id=id
        self.leftthismin_avg=0
        self.leftthismin=0
        self.minlanes,self.maxlanes=laneboundaries
    def __str__(self): 
        return f"edge {self.id} cars:{np.sum(self.cars[:,0])}"
    def __repr__(self): 
        return f"edge {self.id} cars:{np.sum(self.cars[:,0])}"
    def C(self): #capacity of edge
        return int(self.lanes*self.length/(4.5+self.speed/1.8))
    def tto(self):
        return self.length/self.speed*60/1000
    def update(self):
        Filter = np.nonzero(self.cars[:,0]==1)[0] #Filter for 
        self.cars[Filter,2] -= 1 #travelled one minute
        self.cars[Filter,1] += 1 #aged one minute
        if self.size()>0.9*self.C():
            if self.lanes<self.maxlanes:
                print(f"spitsstrook open op {self}")
                self.lanes+=1
                self.cars=np.concatenate((self.cars,np.zeros((self.C()-len(self.cars),5))))
            elif self.size()>0.9*self.Cmax and self.speed>0.5*self.basespeed:
                print(f"file op {self},speed is now {self.speed}")
                self.speed-=.1*self.basespeed
                self.cars=np.concatenate((self.cars,np.zeros((self.C()-len(self.cars),5))))
        self.leftthismin_avg=(1-smoothing)*self.leftthismin_avg+smoothing*self.leftthismin
        self.leftthismin=0
    def tt_avg(self):
        if np.sum(self.cars[:,0])==0: return self.tto()*0.1 #defaults to tt0 if road is empty
        else: return (max(np.sum(self.cars[self.cars[:,2]>0,2]),1)/np.sum(self.cars[:,0])) #otherwise average the traveltime
    def avg_waittime(self):
        """average time a car waits to pass through node"""
        if np.sum(self.cars[:,2]<0)==0: return 0
        else: return -np.average(self.cars[self.cars[:,2]<0,2])
    def tt(self):
        return self.tto()*(1+0.15*(self.size()/self.C())**4)
    def size(self):
        return sum(self.cars[:,0])
class node(object):
    def __init__(self,inputs=[],output=[],destinations=[],id=0,average=85,sd=1.5):
        self.id=id
        self.output = output
        self.input = inputs
        self.destinations=destinations
        self.dest_weights=[]
        self.waitlist=dict(zip(self.output,[dict(zip(self.input,[np.zeros(0,dtype=int) for j in self.input]))for i in self.output]))
        for i in self.waitlist.keys():
            self.waitlist[i]["self"]=np.array([],dtype=int)
        self.buildup=np.zeros((0,5))
        self.times=np.zeros((0,2)) #array with first collumn as times, and second as origins
        self.gen_average=average
        self.gen_sd=sd
    def initwaitlist(self):
        if len(self.waitlist.keys())>0: print("waitlist already had keys",self.waitlist); return
        else: self.waitlist=dict(zip(self.output,[dict(zip(self.input,[np.zeros(0,dtype=int) for j in self.input]))for i in self.output]))
    def __str__(self): 
        if len(self.buildup): return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]} and buildup:{len(self.buildup)}"
        else: return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]}"
    def __repr__(self): 
        if len(self.buildup): return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]} and buildup:{len(self.buildup)}"
        else: return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]}"
    def size(self):
        return len(self.buildup)    
    def avg_waittime(self,edge_out):
        if edge_out not in self.output: print(f"what ya doing mate {edge_out} does not exit {self}")
        carswaiting=np.concatenate([i.cars[self.waitlist[edge_out][i],2] for i in self.input])
        if len(carswaiting)==0: return 0.00001
        else:return -np.average(carswaiting)
    def flow(self):
        for i in self.input: #putting stuff in waitlist
            Remove=np.nonzero(np.logical_and(i.cars[:,4]==self.id,np.logical_and(i.cars[:,2]<=0,i.cars[:,0]==1)))[0]
            self.times=np.concatenate((self.times,i.cars[Remove,:][:,[1,3]])) #remove all cars that have arrived at their destination
            i.cars[Remove,:]=0
            i.leftthismin+=len(Remove)
            if self.output==[]:print(f"node {self.id} had no output roads") ;continue
            Filter2 = np.nonzero(np.logical_and(i.cars[:,2]==0,i.cars[:,0]==1))[0] #check which cars arrived now
            if len(self.output)==len(self.input)==2: choices=np.full(len(Filter2),[edges for edges in self.output if edges.id !=(i.id[1],i.id[0])][0]) #cars are not allowed to go back, so this is the only option for cars
            else:
                handling=i.cars[Filter2,:] #cars that need to be handled this function call
                choices=np.empty(len(handling),dtype=edge)
                for dest in np.unique(handling[:,4]):
                    routes=list(islice(nx.shortest_simple_paths(gfd,self.id,dest,weight="tt_avg"),len(self.output)-1))
                    a=[gfd.edges[path[:2]]["object"]for path in routes]
                    weights=1/np.array([nx.path_weight(gfd,path,weight="tt_avg") for path in routes])
                    rander=np.random.choice(a,size=sum(handling[:,4]==dest),p=weights/np.sum(weights)) #randomly choose next edge
                    choices[handling[:,4]==dest]=rander
            for indexes in choices:
                if indexes not in self.output: print("fuck")
            for j in self.output:
                self.waitlist[j][i]=np.concatenate((self.waitlist[j][i],np.array(Filter2[choices==j],dtype=int))) #add new indexes along with their location at the end
        if len(self.buildup)>0: #handle buildup caused by generate
            if len(self.output)==1: choices=np.full(len(self.buildup),self.output[0])
            elif len(self.output)==len(self.input)==2: choices=np.full(len(Filter2),[edges for edges in self.output if edges.id !=(i.id[1],i.id[0])][0]) #cars are not allowed to go back, so this is the only option for cars
            else:   
                choices=np.empty(len(self.buildup),dtype=edge)
                for dest in np.unique(self.buildup[:,4]):
                    routes=list(islice(nx.shortest_simple_paths(gfd,self.id,dest,weight="tt_avg"),len(self.output)-1))
                    a=[gfd.edges[path[:2]]["object"]for path in routes]
                    weights=1/np.array([nx.path_weight(gfd,path,weight="tt_avg") for path in routes])
                    rander=np.random.choice(a,size=sum(self.buildup[:,4]==dest),p=weights/np.sum(weights)) #randomly choose next edge
                    choices[self.buildup[:,4]==dest]=rander
            for o in self.output:
                self.waitlist[o]["self"]=np.nonzero(choices==o)[0]
        for i in self.output:#getting stuff out of waitlist
            if sum([len(self.waitlist[i][j]) for j in self.input])+len(self.waitlist[i]["self"])==0: continue #doesn't need to do anything if there are no cars waiting
            elif sum([len(self.waitlist[i][j]) for j in self.input])+len(self.waitlist[i]["self"])<=np.sum(i.cars[:,0]==0):
                for j in self.input:
                    filter=np.nonzero(i.cars[:,0]==0)[0][:len(self.waitlist[i][j])]
                    j.leftthismin+=len(filter)
                    i.cars[filter]=j.cars[self.waitlist[i][j],:] #put cars in next edge
                    i.cars[filter,2]=np.maximum(np.ceil(i.tt()+np.random.normal(0,i.tto()/10,size=len(filter))),1)
                    j.cars[self.waitlist[i][j],:]=0 #empty the cars
                    self.waitlist[i][j]=np.zeros(0,dtype=int) #empty waitlist
                filter=np.nonzero(i.cars[:,0]==0)[0][:len(self.waitlist[i]["self"])]
                i.cars[filter]=self.buildup[self.waitlist[i]["self"],:]
                i.cars[filter,2]=np.maximum(np.ceil(i.tt()+np.random.normal(0,i.tto()/10,size=len(filter))),1)
                self.buildup[self.waitlist[i]["self"],0]=0
            else:
                space=np.sum(i.cars[:,0]==0)
                needs=np.array([len(self.waitlist[i][j]) for j in self.input]+[len(self.waitlist[i]["self"])])
                allowances=distribute(needs,space)
                for k,j in enumerate(self.input):
                    j.leftthismin+=allowances[k]
                    filter=np.nonzero(i.cars[:,0]==0)[0][:allowances[k]]
                    i.cars[filter]=j.cars[self.waitlist[i][j][:allowances[k]]] ###probs dont need [filtertje,:] [filtertje] seems enough
                    i.cars[filter,2]=np.maximum(np.ceil(i.tt()+np.random.normal(0,i.tto()/10,size=len(filter))),1)
                    j.cars[self.waitlist[i][j][:allowances[k]],:]=0 #empty the cars
                    self.waitlist[i][j]=self.waitlist[i][j][allowances[k]:]
                #handle buildup
                filter=np.nonzero(i.cars[:,0]==0)[0][:allowances[-1]]
                i.cars[filter]=self.buildup[self.waitlist[i]["self"][:allowances[-1]]] ###probs dont need [filtertje,:] [filtertje] seems enough
                i.cars[filter,2]=np.maximum(np.ceil(i.tt()+np.random.normal(0,i.tto()/10,size=len(filter))),1)
                self.buildup[self.waitlist[i]["self"][:allowances[-1]],0]=0 #empty the cars
    def generate(self,destinations=[2]): ##change code to account for more generating cities and differing distributions
        N=int(np.random.normal(self.gen_average,self.gen_sd)+0.5) #+.5 rounds it to the nearest integer
        self.buildup=self.buildup[self.buildup[:,0]!=0]
        newcars=np.array([np.ones(N),np.zeros(N),np.random.normal(0,1,size=N),np.full(N,self.id),np.random.choice(self.destinations,N,p=self.dest_weights)]).transpose()
        self.buildup=np.concatenate((self.buildup,newcars)) #needs to handle the built up cars first
def carspermin(workday_avg,part_during_day=.8):return workday_avg*part_during_day/12/60
def compare_data(edges,counter_id,workday_avg,name,part_during_day=.8):
    counts=[]
    for i in edges:
        if len(loaded_gpd[(loaded_gpd["node_start"]==i[0])*(loaded_gpd["node_end"]==i[1])]["leftforw"])>=1: counts.append(loaded_gpd[(loaded_gpd["node_start"]==i[0])*(loaded_gpd["node_end"]==i[1])]["leftforw"].iloc[0])
        else: counts.append(loaded_gpd[(loaded_gpd["node_start"]==i[1])*(loaded_gpd["node_end"]==i[0])]["leftback"].iloc[0])
    line=f"counter {counter_id}:{carspermin(workday_avg,part_during_day)}, model: {sum(counts)} {counts} name:{name}"
    print(line)
    return carspermin(workday_avg,part_during_day)-sum(counts),line
def test_Aweg(lines_gpd,wegnr,measurement):
    """compares the average of the flow of all roadsegments of a highway and compares it with an hourly measurement on the real road"""
    model,real=np.average(lines_gpd["leftback"][lines_gpd["wegnr"]==wegnr])+np.average(lines_gpd["leftforw"][lines_gpd["wegnr"]==wegnr]),measurement/60
    line=f"A{wegnr}:{real},{model=}"
    return real-model, line
base_nodes_gpd = gpd.read_file('./wegengraaf2copy/points.shp')
base_lines_gpd = gpd.read_file('./wegengraaf2copy/lines.shp')
base_lines_gpd["capacity"]=base_lines_gpd['length']*base_lines_gpd['nbanen']*1/(4.5+base_lines_gpd['maxshd']/1.8)
base_nodes_gpd["is_auto_br"][np.isnan(base_nodes_gpd["is_auto_br"])]=0
base_nodes_gpd["spawn_hoev"][np.isnan(base_nodes_gpd["spawn_hoev"])]=0
nodes_gpd,lines_gpd=base_nodes_gpd.copy(),base_lines_gpd.copy()
nodes_gpd["spawn_hoev"][nodes_gpd["nodeID"]==160],nodes_gpd["is_auto_br"][nodes_gpd["nodeID"]==160]=      spawn_hoeveelheden[160],1
nodes_gpd["spawn_hoev"][nodes_gpd["nodeID"]==199],nodes_gpd["is_auto_br"][nodes_gpd["nodeID"]==199]=      spawn_hoeveelheden[199],1 #afsluitdijk
for name in stadnamen:
    nodes_gpd["spawn_hoev"][nodes_gpd["stad_naam"]==name]=spawn_hoeveelheden[name]
def create_graph(nodes_gpd, lines_gpd):
    gf = nx.Graph()
    for _, k in nodes_gpd.iterrows():
        gf.add_node(int(k['nodeID']),
                    x=k['geometry'].x,
                    y=k['geometry'].y,
                    stad_naam=k['stad_naam'],
                    is_auto_br=k['is_auto_br'],
                    spawn_hoev=k['spawn_hoev'])
    for _, row in lines_gpd.iterrows():
        gf.add_edge(int(row['node_start']), int(row['node_end']),
                    wegnr=row['wegnr'],
                    length=row['length'],
                    maxshd=row['maxshd'],
                    nbanen=row['nbanen'])
    return gf

gfd = nx.DiGraph(create_graph(nodes_gpd, lines_gpd))
def weight_edge_min(f, t, attr):
    return attr['length']/attr['maxshd']/1000*60

def weight_edge_dist(f, t, attr):
    return attr['length']

def weight_capacity(f, t, attr):
    cars = np.sum(attr['object'].cars[:, 0])
    capacity = attr['object'].C()
    return attr['length']/attr['maxshd']/1000*60 * (1 + cars/capacity)
gfd = nx.DiGraph(create_graph(nodes_gpd, lines_gpd))
spawns={int(nodes_gpd["nodeID"][nodes_gpd["stad_naam"]==name]):spawn_hoeveelheden[name] for name in list(spawn_hoeveelheden.keys())[2:]}
spawns[160],spawns[199]=spawn_hoeveelheden[160],spawn_hoeveelheden[199]

C:\Users\kdvan\AppData\Local\Temp\ipykernel_26220\4189124253.py:194: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  base_nodes_gpd["is_auto_br"][np.isnan(base_nodes_gpd["is_auto_br"])]=0
C:\Users\kdvan\AppData\Local\Temp\ipykernel_26220\41891

In [6]:
list_of_vals[1::4]
# for i in list_of_vals: 108,105,110,107
#     if i not in list_of_vals[0::4]+list_of_vals[1::4]+list_of_vals[2::4]+list_of_vals[3::4]: print(i)

[(1,
  {160: 12,
   199: 25,
   'Leeuwarden': 56,
   'Drachten': 30,
   'Sneek': 35,
   'Harlingen': 13.086859668020224,
   'Joure': 10,
   'Wolvega': 11,
   'Franeker': 10,
   'Dokkum': 11,
   'Lemmer': 14,
   'Bolsward': 9.477514803509319,
   'Burgum': 10,
   'Heerenveen': 24}),
 (5,
  {160: 12,
   199: 25,
   'Leeuwarden': 56,
   'Drachten': 36,
   'Sneek': 35,
   'Harlingen': 13.086859668020224,
   'Joure': 10,
   'Wolvega': 11,
   'Franeker': 10,
   'Dokkum': 11,
   'Lemmer': 14,
   'Bolsward': 9.477514803509319,
   'Burgum': 10,
   'Heerenveen': 24}),
 (9,
  {160: 20,
   199: 25,
   'Leeuwarden': 56,
   'Drachten': 30,
   'Sneek': 35,
   'Harlingen': 13.086859668020224,
   'Joure': 10,
   'Wolvega': 11,
   'Franeker': 10,
   'Dokkum': 11,
   'Lemmer': 14,
   'Bolsward': 9.477514803509319,
   'Burgum': 10,
   'Heerenveen': 24}),
 (13,
  {160: 20,
   199: 25,
   'Leeuwarden': 56,
   'Drachten': 36,
   'Sneek': 35,
   'Harlingen': 13.086859668020224,
   'Joure': 10,
   'Wolvega': 11

In [7]:
for runid,newspawns in list_of_vals[108::4]:
    #adjust spawnrates
    nx.set_node_attributes(gfd,newspawns,name="spawn_hoev")
    for ID in newspawns.keys():
        nodes_gpd.loc[nodes_gpd["nodeID"]==ID,"spawn_hoev"]=newspawns[ID]
    #put objects on network
    sources=nodes_gpd["nodeID"][np.nonzero(nodes_gpd["is_auto_br"])[0]]
    weights=nodes_gpd["spawn_hoev"][np.nonzero(nodes_gpd["is_auto_br"])[0]]
    results=[]
    for trial in range(1,5):
        nx.set_edge_attributes(gfd,{e: edge(gfd.edges[e]["length"],gfd.edges[e]["maxshd"],gfd.edges[e]["nbanen"],id=e) for e in gfd.edges},"object") #put edge objects on the edges
        nx.set_node_attributes(gfd,{n: node(inputs=[gfd.edges[e]["object"] for e in gfd.in_edges(n)],output=[gfd.edges[e]["object"] for e in gfd.out_edges(n)],id=n, average=gfd.nodes[n]["spawn_hoev"],sd=1.5) for n in gfd.nodes},"object") 
        #put node objects on nodes
        for i in sources:
            gfd.nodes[i]["object"].destinations=list(sources[sources!=i])
            gfd.nodes[i]["object"].dest_weights=list(weights[sources!=i]/sum(weights[sources!=i])) #destination is any city except source
        #running the simulation    
        minutes_init=0 #initialisation time
        minutes_sim=300 #simulation time
        def tt_avg(f, t, attr):
            return attr['object'].tt_avg()+attr['object'].avg_waittime() #average waittime is remaning time plus wait time at end of edge
        for minute in range(minutes_init):
            nx.set_edge_attributes(gfd,{e: (gfd.edges[e]["object"].tt_avg()+gfd.nodes[e[0]]["object"].avg_waittime(gfd.edges[e]["object"]))*(1+gfd.edges[e]["object"].size()/gfd.edges[e]["object"].C()) for e in gfd.edges},"tt_avg") #put edge objects on the edges
            paths={n: b for (n, (a, b)) in nx.all_pairs_dijkstra(gfd, weight="tt_avg")}
            for i in sources:
                # print(i,gfd.nodes[i]["object"])
                gfd.nodes[i]["object"].generate() #generate cars at all sources
            for e in gfd.edges:
                # print(e,gfd.edges[e]["object"])
                gfd.edges[e]["object"].update()
            for n in gfd.nodes:
                # print(n,gfd.nodes[n]["object"])
                gfd.nodes[n]["object"].flow()
        for i in sources: gfd.nodes[i]["object"].times=np.zeros((0,2))
        for minute in range(minutes_sim):
            print(minute, end=', ')
            nx.set_edge_attributes(gfd,{e: (gfd.edges[e]["object"].tt_avg()+gfd.nodes[e[0]]["object"].avg_waittime(gfd.edges[e]["object"]))*(1+gfd.edges[e]["object"].size()/gfd.edges[e]["object"].C()) for e in gfd.edges},"tt_avg") #put edge objects on the edges
            paths={n: b for (n, (a, b)) in nx.all_pairs_dijkstra(gfd, weight="tt_avg")}
            for i in sources:
            # print(i,gfd.nodes[i]["object"])
                gfd.nodes[i]["object"].generate() #generate cars at all sources
            for e in gfd.edges:
                # print(e,gfd.edges[e]["object"])
                gfd.edges[e]["object"].update()
            for n in gfd.nodes:
                # print(n,gfd.nodes[n]["object"])
                gfd.nodes[n]["object"].flow()
        #checking howmany cars left each minute for all edges
        arr_forw=np.zeros(len(lines_gpd))
        arr_backw=np.zeros(len(lines_gpd))
        for index,(start,end) in enumerate(zip(lines_gpd["node_start"],lines_gpd["node_end"])):
            # if (start,end) in gfd.edges:
            arr_forw[index]=gfd.edges[(start,end)]["object"].leftthismin_avg
            arr_backw[index]=gfd.edges[(end,start)]["object"].leftthismin_avg
        loaded_gpd=pd.concat((lines_gpd,pd.DataFrame(arr_forw,columns=["leftforw"]),pd.DataFrame(arr_backw,columns=["leftback"])),axis=1) #naming convention I cannot be arsed to change, used in compare_data
        #creating a text file
        f=open(f"testresults\\test{runid}_{trial}","w")
        f.write(f"trial {trial} with settings:{newspawns}\n")
        counters2=[([(202,35),(35,202)],101608,24097,.807,.5,"oost Leeuwarden"),([(72,173),(173,72)],101625,8858,.815,1,"leeuwarden ZW"),([(205,206),(206,205),(204,205),(205,204)],(101612,101647),19083+7763,.818,1,"bergum ZO/ZW"),([(155,205),(205,155)],101615,18749,.813,1,"Bergum NW"),([(240,241),(241,240)],101606,11187,.842,1,"sneek NO"),([(164,82),(82,164)],101607,10613,.826,1,"sneek Z"),([(502,503),(503,502)],101650,4393,.805,1,"Heerenveen Zuid") \
          ,([(144,172),(172,144)],101673,9766,.798,1,"Drachten N"),([(155,63),(63,155)],101614,14396,.822,1,"Dokkum Z"),([(176,63),(63,176)],101671,9950,.844,1,"Dokkum W"),([(180,207),(207,180)],101601,9822,.817,1,"Wolvega O"),([(71,102),(102,71)],101668,11147,.811,1,"Lemmer W"),([(136,134),(134,136)],101676,6883,.829,1,"Harlingen O"),(([(136,16),(16,136)],101643,2707,.798,1,"Harlingen N")),\
            ([(137,14),(14,137)],101637,3407,.798,1,"FranekerN"),([(75,45),(45,75)],101636,2948,.834,1,"Franeker ZO"),([(57,152),(152,57)],101623,8660,.816,1,"Bolsward Z"),([(115,77),(77,115)],101624,8445,.816,1,"Bolsward N"),([(219,160),(160,219),(178,160),(160,178)],(101674,101656),16267+5077,.828,1,"Oosterwolde N/W")]
        sq_sum=0
        for edges,counter_id,workday_avg,part_during_day,weight,name in counters2:
            dist,line=compare_data(edges,counter_id,workday_avg,name,part_during_day)
            f.write(line+"\n")
            sq_sum+=weight*dist**2
        wegnr,measurement=("031",(894+877+978)/3)
        dist,line=test_Aweg(loaded_gpd,wegnr,measurement)
        f.write(line+"\n")
        new_dist=np.sqrt(sq_sum)
        results.append(new_dist)
        f.write(f"{new_dist}")
        f.close()
    avg,sd=np.average(results),np.std(results)
    f=open(f"totalresults_test{runid}","w")
    f.write(f"{newspawns}\n {avg=},{sd=}")
    f.close()
    new_dist=np.average(results)


0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, file op edge (87, 85) cars:18.0,speed is now 80
13, 14, 15, 16, 17, 18, 19, 20, 21, 22, file op edge (500, 133) cars:9.0,speed is now 60
23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, file op edge (174, 173) cars:21.0,speed is now 80
file op edge (1003, 149) cars:50.0,speed is now 50
44, 45, 46, file op edge (174, 173) cars:23.0,speed is now 72.0
47, 48, 49, file op edge (174, 173) cars:27.0,speed is now 64.0
50, file op edge (215, 151) cars:24.0,speed is now 100
file op edge (1000, 6) cars:123.0,speed is now 50
51, file op edge (149, 150) cars:111.0,speed is now 100
52, 53, file op edge (149, 150) cars:120.0,speed is now 90.0
54, 55, 56, file op edge (174, 173) cars:28.0,speed is now 56.0
57, file op edge (174, 173) cars:34.0,speed is now 48.0
58, 59, 60, 61, 62, 63, 64, 65, 66, file op edge (206, 146) cars:115.0,speed is now 100
67, 68, 69, file op edge (206, 146) cars:127.0,speed is now 90.0
file op edge 